In [11]:
import numpy as np
import lmfit

from General.experiments.hdf5.readHDF5 import read_hdf5
from General.experiments.waveforms import MeasuredWaveforms
from General.plotting import plot, cbar
from General.plotting import linestyles
import General.numpy_funcs as npf

out_loc = r'E:\OneDrive - TU Eindhoven\Master thesis\Tex\Images\Method'

In [13]:
bloc = r'E:\OneDrive - TU Eindhoven\Master thesis\Results\Air_3slm_Ar_1slm_9kV_'
for t in (0.3, 0.5, 1, 2,):
    loc = rf'{bloc}{t}us.hdf5'
    data = read_hdf5(loc)
    waveform_data = data['waveforms']
    waveforms = MeasuredWaveforms.from_waveforms(waveform_data, channels={1: 'voltage', 2: 'current', 3: 'pulse_generator', 4: 'ground_current'})
    
    is_on = waveforms.is_on()
    num = np.sum(is_on.astype(int))
    values = np.argwhere(np.diff(is_on.astype(int)) == 1)[0][0] + np.linspace(10, num-52, 10, dtype=int)
    indexes = values + np.arange(20, dtype=int)[:, None]
    
    times = np.mean(waveforms.time[indexes], axis=0)
    currents = np.mean(waveforms.currents[indexes], axis=0)
    rel_times = waveforms.time_offset[values]
    rel_times = rel_times - rel_times[0]
    colors, _ = cbar.cbar_norm_colors(rel_times)
    labels = [f'{rel_time/60:.0f}' for rel_time in rel_times]
    
    plot_kwargs = {'xlabel': 'Time [us]', 'ylabel': 'Current [A]', 'xlim': (0, 0.7+t)}
    legend_kwargs= {'title': 'Time [min]', 'loc': 'upper right'}
    save_loc = rf'{out_loc}\currents_{t}.pdf'
    
    l_kwargs = linestyles.legend_linelooks_by(color_labels=labels, color_values=colors, color_title='Time [min]')
    
    plot.lines(1e6*times, currents, colors=colors, labels=labels, plot_kwargs=plot_kwargs, legend_kwargs=legend_kwargs, save_loc=save_loc)
    plot.lines(1e6*times, currents, colors=colors, labels=labels, plot_kwargs=plot_kwargs, legend_kwargs=legend_kwargs, save_loc=save_loc.replace('.pdf', '_l.pdf'))
    
    max_time = 1e6*times[1][np.argmax(currents[1])]
    min_time = 1e6*times[1][np.argmin(currents[1])]
    dt =min_time - max_time
    
    save_loc = rf'{out_loc}\currents_{t}2.pdf'
    fig, ax = plot.lines(1e6*times, currents - currents[0], colors=colors, labels=labels, plot_kwargs=plot_kwargs, legend_kwargs=legend_kwargs, show=False)
    ax.axvline(max_time, color='black', linestyle='-')
    ax.axvline(min_time, color='black', linestyle='-')
    if t == 0.3:
        ax.axvline(max_time + 0.125, color='black', linestyle=':')
        ax.axvline(min_time - 0.125, color='black', linestyle=':')
    else:
        ax.axvline(max_time + 0.25, color='black', linestyle=':')
        ax.axvline(min_time - 0.15, color='black', linestyle=':')
    fig.savefig(save_loc)
    
    n = 100
    _, f_val, f_std = waveforms.background_current_fitting(t, block_average=n)
    on_mask = waveforms.is_on()
    a_val, a_std = waveforms.background_current_averaging(on_mask=on_mask, block_average=n)
    a_val2, a_std2 = waveforms.background_current_averaging(on_mask=on_mask, block_average=n, start_offset=0.45, end_offset=0.45)
    times = 1e6*npf.block_average(waveforms.time[on_mask], n)
    currents = npf.block_average(waveforms.currents[on_mask], n)
    start_time = np.average(times, axis=0)[np.argmax(np.average(currents, axis=0))]
    end_time = np.average(times, axis=0)[np.argmin(np.average(currents, axis=0))]
    s_time = start_time + (end_time - start_time)/3
    e_time = end_time - (end_time - start_time)/3
    
    plot_kwargs = {'xlabel': 'Time [us]', 'ylabel': 'Current [A]', 'xlim': (0.375, 0.19+t)}
    fig, ax = plot.lines(times, currents, show=False, plot_kwargs=plot_kwargs)
    ax.axvline(s_time, color='black', linestyle=':')
    ax.axvline(e_time, color='black', linestyle=':')
    for i, (value, value2, value3) in enumerate(zip(f_val, a_val, a_val2)):
        ax.axhline(value, color=f'C{i}', linestyle='--')
        ax.axhline(value2, color=f'C{i}', linestyle='-')
        ax.axhline(value3, color=f'C{i}', linestyle=':')

plot.export_legend(l_kwargs, rf'{out_loc}\time_legend.pdf')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
is_on = waveforms.is_on()
num = np.sum(is_on.astype(int))
values = np.argwhere(np.diff(is_on.astype(int)) == 1)[0][0] + np.linspace(10, num-52, 10, dtype=int)
indexes = values + np.arange(20, dtype=int)[:, None]

times = np.mean(waveforms.time[indexes], axis=0)
currents = np.mean(waveforms.currents[indexes], axis=0)
rel_times = waveforms.time_offset[values]
rel_times = rel_times - rel_times[0]
colors, _ = cbar.cbar_norm_colors(rel_times)
labels = [f'{rel_time/60:.0f}' for rel_time in rel_times]

plot_kwargs = {'xlabel': 'Time [us]', 'ylabel': 'Current [A]', 'xlim': (0, 2)}
legend_kwargs= {'title': 'Time [min]', 'loc': 'upper right'}
save_loc = rf'{out_loc}\currents.pdf'
plot.lines(1e6*times, currents, colors=colors, labels=labels, plot_kwargs=plot_kwargs, legend_kwargs=legend_kwargs, save_loc=save_loc)

In [ ]:
start_time = np.average(waveforms.time, axis=0)[np.argmax(np.average(waveforms.currents, axis=0))] - 1e-7
end_time = np.average(waveforms.time, axis=0)[np.argmin(np.average(waveforms.currents, axis=0))] - 1e-7

vals, stds = waveforms.background_current_averaging()
vals, stds = vals[values], stds[values]

mask = (waveforms.time[0] > end_time - 1e-6/3) & (waveforms.time[0] < end_time)
model = waveforms._background_current_fitting_model
lmfit_model = lmfit.Model(model)
lmfit_model.set_param_hint('amplitude', value=0.25, min=0)
lmfit_model.set_param_hint('decay', value=2e-6)
lmfit_model.set_param_hint('phase', value=-1.5)
lmfit_model.set_param_hint('length', value=1.9e-7)
lmfit_model.set_param_hint('offset', value=0.8)
params = lmfit_model.make_params()

vals2 = np.zeros_like(vals)
for index, (tim, current) in enumerate(zip(times, currents)):
    result = lmfit_model.fit(current[mask], params, x=tim[mask])
    vals2[index] = result.best_values['offset']
    params = result.params

plot_kwargs = {'xlabel': 'Time [us]', 'ylabel': 'Current [A]', 'xlim': (0.4, t)}
legend_kwargs = {'loc': 'center left', 'bbox_to_anchor': (1, 0.5), 'title':'Time [min]'}
fig, ax = plot.lines(1e6*times, currents, colors=colors, labels=labels, plot_kwargs=plot_kwargs, legend_kwargs=legend_kwargs, show=False)
ax.axvline(end_time*1e6, color='black', linestyle='--')
ax.axvline((end_time)*1e6-(1/3), color='black', linestyle='--')
for i, (value, std) in enumerate(zip(vals, stds)):
    ax.axhline(value, color=colors[i], linestyle=':')
    ax.axhline(vals2[i], color=colors[i], linestyle='--')
fig.show()

print(start_time, end_time)